In [29]:
!rm $DL/*

rm: /Users/damiansp/Downloads/*: No such file or directory


In [80]:
from datetime import datetime

import numpy as np
import pandas as pd

In [81]:
SYMBOLS = [
    'AAPL', 'ABMD', 'ABNB', 'AMD', 'AMZN', 'ANET', 'APPH', 'AXON',
    'BROS', 'BYND', 'CHWY', 'COIN', 'COUR', 'CRWD', 'DDOG', 'DKNG',
    'DOCU', 'DUOL', 'EDIT', 'ETSY', 'FLGT', 'FUV', 'FVRR', 'GH', 'GMED',
    'GOOG', 'INTG', 'ISRG', 'JD', 'LSPD', 'LULU','MDB', 'MELI', 'MNST',
    'MSFT', 'MTCH', 'NVCR', 'NVDA', 'NYT', 'OKTA', 'OPEN', 'PANW',
    'PINS', 'PTON', 'QDEL', 'RBLX', 'RDFN', 'RGEN', 'ROKU', 'SE',
    'SHOP', 'SPLK', 'SQ', 'TASK', 'TDOC', 'TEAM', 'TREX', 'TTD', 'TTWO',
    'TWLO', 'TWTR', 'UPST', 'VEEV', 'WK', 'XPEV', 'ZEN', 'ZNGA']

In [82]:
DL = '/Users/damiansp/Downloads'

In [83]:
!ls $DL

Holdings - Damian Satterthwaite-Phillips.csv


In [84]:
def check_symbols(df):
    all_present = True
    for symbol in df.index.unique():
        if symbol not in SYMBOLS:
            print(f'{symbol} not in SYMBOLS')
            all_present = False
    if all_present:
        print('All clear')

In [85]:
def convert_value(s):
    return round(float(s.replace('$', '').replace(',', '')))

In [86]:
binder = pd.DataFrame(index=SYMBOLS)
binder.head()

""
AAPL
ABMD
ABNB
AMD
AMZN


### E*Trade

In [63]:
DONGMEI = False

In [64]:
if DONGMEI:
    et_df = (
        pd.read_csv(
            f'{DL}/Positions.csv', index_col=0, skiprows=1
        )[['Market Value']]
        .rename(columns={'Market Value': 'ET'}))
    et_df.index = map(lambda x: x.split()[0], et_df.index)
    et_df.drop(index=['Portfolio', 'Cash'], inplace=True)
else:
    et_df = (
        pd.read_csv(
            f'{DL}/Positions.csv', index_col=0, skiprows=1
        )[['Market Value']]
        .rename(columns={'Market Value': 'ET'}))
    et_df.index = map(lambda x: x.split()[0], et_df.index)
    et_df.drop(index=['FNRG', 'Portfolio', 'Cash'], inplace=True)
    #et_df.ET = et_df.ET.apply(lambda x: round(float(x)))
et_df.head()

,ET
ABMD,--
ABNB,151.8
AMZN,2936.35
ANET,241.82
CHWY,383.7


In [68]:
#et_df.loc['ABMD', 'ET'] = 305.62

In [69]:
check_symbols(et_df)

All clear


In [70]:
out = pd.concat([binder, et_df], axis=1).fillna(0)
out.ET = out.ET.astype(float).apply(lambda x: int(round(x)))
out.head()

,ET
AAPL,0
ABMD,306
ABNB,152
AMD,0
AMZN,2936


### Fidelity

In [71]:
TODAY = datetime.now().date()
TODAY
today = datetime.strftime(TODAY, '%b-%d-%Y')
filename = f'Portfolio_Positions_{today}.csv'
filename

'Portfolio_Positions_Mar-10-2022.csv'

In [72]:
df_raw = pd.read_csv(f'{DL}/{filename}')
df_raw.head()

,Account Number,Account Name,Symbol,Description,Quantity,Last Price,Last Price Change,Current Value,Today's Gain/Loss Dollar,Today's Gain/Loss Percent,Total Gain/Loss Dollar,Total Gain/Loss Percent,Percent Of Account,Cost Basis,Cost Basis Per Share,Type
0,218520300,ROLLOVER IRA,SPAXX**,FIDELITY GOVERNMENT MONEY MARKET,19297.59,$1.00,$0.00,$19297.59,NaN,NaN,NaN,NaN,45.10%,NaN,NaN,Cash
1,218520300,ROLLOVER IRA,AMD,ADVANCED MICRO DEVICES INC,9.00,$106.46,-$4.59,$958.14,-$41.31,-4.14%,+$44.01,+4.81%,2.24%,$914.13,$101.57,Cash
2,218520300,ROLLOVER IRA,CHWY,CHEWY INC CL A,6.00,$38.37,-$3.28,$230.22,-$19.68,-7.88%,-$26.04,-10.17%,0.54%,$256.26,$42.71,Cash
3,218520300,ROLLOVER IRA,DOCU,DOCUSIGN INC COM,29.00,$93.88,-$4.17,$2722.52,-$120.93,-4.26%,-$524.61,-16.16%,6.36%,$3247.13,$111.97,Cash
4,218520300,ROLLOVER IRA,GH,GUARDANT HEALTH INC COM,13.00,$58.68,-$1.61,$762.84,-$20.93,-2.68%,-$45.77,-5.67%,1.78%,$808.61,$62.20,Cash


In [73]:
df = df_raw[['Account Name', 'Symbol', 'Current Value']].dropna()
df['Current Value'] = df['Current Value'].apply(convert_value)
df['Current Value'] = df['Current Value'].fillna(0).astype(int)
df = df[((df.Symbol != 'SPAXX**') & (df.Symbol != 'Pending Activity'))]
df.head()

,Account Name,Symbol,Current Value
1,ROLLOVER IRA,AMD,958
2,ROLLOVER IRA,CHWY,230
3,ROLLOVER IRA,DOCU,2723
4,ROLLOVER IRA,GH,763
5,ROLLOVER IRA,GMED,282


In [74]:
accounts = {account_name: df[df['Account Name'] == account_name]
            for account_name in df['Account Name'].unique()}
for acct in accounts.keys():
    accounts[acct].index = accounts[acct].Symbol
    accounts[acct] = (
        accounts[acct]
        .rename(columns={'Current Value': acct})
        .drop(columns=['Account Name', 'Symbol']))
    
accounts['ROLLOVER IRA'].head()

,ROLLOVER IRA
Symbol,
AMD,958
CHWY,230
DOCU,2723
GH,763
GMED,282


In [75]:
for k, v in accounts.items():
    print(k)
    check_symbols(v)
    print()

ROLLOVER IRA
All clear

ROTH IRA
All clear

SIMPLE IRA
All clear



In [76]:
for v in accounts.values():
    out = pd.concat([out, v], axis=1)
out.fillna(0, inplace=True)
out = out.astype(int)
out.head()

,ET,ROLLOVER IRA,ROTH IRA,SIMPLE IRA
AAPL,0,0,1268,0
ABMD,306,0,611,0
ABNB,152,0,152,0
AMD,0,958,0,0
AMZN,2936,0,0,0


In [77]:
out.to_csv(f'{DL}/clean.csv')

In [78]:
!open $DL/clean.csv

In [79]:
!rm $DL/$filename $DL/Positions.csv $DL/clean.csv
!ls $DL

### MarketWatch

In [100]:
!ls $DL

Holdings - Damian Satterthwaite-Phillips.csv


In [101]:
mw = pd.read_csv(f'{DL}/Holdings - Damian Satterthwaite-Phillips.csv')[['Symbol', 'Value']]
mw.index = mw.Symbol
mw.drop('Symbol', axis=1, inplace=True)
mw.Value = mw.Value.apply(convert_value)
mw.head()

,Value
Symbol,
ABMD,2868
AMD,1773
AXON,1621
BROS,2245
BYND,620


In [102]:
check_symbols(mw)

All clear


In [103]:
out = pd.concat([binder, mw], axis=1).fillna(0).astype(int)
out.head()

,Value
AAPL,0
ABMD,2868
ABNB,0
AMD,1773
AMZN,0


In [104]:
out.to_csv(f'{DL}/mw.csv')

In [105]:
!open $DL/mw.csv

In [106]:
!rm $DL/"Holdings - Damian Satterthwaite-Phillips.csv" $DL/mw.csv
!ls $DL

In [26]:
#!rm $DL/*